In [2]:
from keras.layers import Conv2D, UpSampling2D
from keras.models import Sequential
import numpy as np
import tensorflow as tf

import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
import random
import cv2
import os
from tqdm import tqdm
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
good_frames = '/content/drive/MyDrive/Autoencoders/valid_upscaled'
bad_frames = '/content/drive/MyDrive/Autoencoders/valid_blur'

In [ ]:
clean_frames = []
for file in tqdm(sorted(os.listdir(good_frames))):
  if any(extension in file for extension in ['.jpg', 'jpeg', '.png']):
    image = tf.keras.preprocessing.image.load_img(good_frames + '/' + file, target_size=(128,128))
    image = tf.keras.preprocessing.image.img_to_array(image).astype('float32') / 255
    clean_frames.append(image)

clean_frames = np.array(clean_frames)
blurry_frames = []
for file in tqdm(sorted(os.listdir(bad_frames))):
  if any(extension in file for extension in ['.jpg', 'jpeg', '.png']):
    image = tf.keras.preprocessing.image.load_img(bad_frames + '/' + file, target_size=(128,128))
    image = tf.keras.preprocessing.image.img_to_array(image).astype('float32') / 255
    blurry_frames.append(image)

blurry_frames = np.array(blurry_frames)


from keras.layers import Dense, Input
from keras.layers import Conv2D, Flatten
from keras.layers import Reshape, Conv2DTranspose
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.utils.vis_utils import plot_model
from keras import backend as K

seed = 21
random.seed = seed
np.random.seed = seed

y_test = clean_frames;
X_test= blurry_frames;

print(X_test[0].shape)
print(y_test[0].shape)

In [ ]:
good1_frames = '/content/drive/MyDrive/Autoencoders/div2k_hr'
bad1_frames = '/content/drive/MyDrive/Autoencoders/div2k_hr_blur'

clean1_frames = []
for file in tqdm(sorted(os.listdir(good1_frames))):
  if any(extension in file for extension in ['.jpg', 'jpeg', '.png']):
    image = tf.keras.preprocessing.image.load_img(good1_frames + '/' + file, target_size=(128,128))
    image = tf.keras.preprocessing.image.img_to_array(image).astype('float32') / 255
    clean1_frames.append(image)

clean1_frames = np.array(clean1_frames)

blurry1_frames = []
for file in tqdm(sorted(os.listdir(bad1_frames))):
  if any(extension in file for extension in ['.jpg', 'jpeg', '.png']):
    image = tf.keras.preprocessing.image.load_img(bad1_frames + '/' + file, target_size=(128,128))
    image = tf.keras.preprocessing.image.img_to_array(image).astype('float32') / 255
    blurry1_frames.append(image)

blurry1_frames = np.array(blurry1_frames)

y_train = clean1_frames;
X_train= blurry1_frames;


print(X_train[0].shape)
print(y_train[0].shape)


In [ ]:
r = random.randint(0, len(clean_frames)-1)
print(r)
fig = plt.figure()
fig.subplots_adjust(hspace=0.1, wspace=0.2)
ax = fig.add_subplot(1, 2, 1)
ax.imshow(clean_frames[r])
ax = fig.add_subplot(1, 2, 2)
ax.imshow(blurry_frames[r])

z= random.randint(0, len(clean1_frames)-1)
print(z)
fig = plt.figure()
fig.subplots_adjust(hspace=0.1, wspace=0.2)
ax = fig.add_subplot(1, 2, 1)
ax.imshow(y_train[z])
ax = fig.add_subplot(1, 2, 2)
ax.imshow(blurry1_frames[z])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, UpSampling2D, PReLU

model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same', strides=2, input_shape=(128, 128, 3)))
model.add(PReLU())
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(PReLU())
model.add(Conv2D(128, (3, 3), padding='same', strides=2))
model.add(PReLU())
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(PReLU())
model.add(Conv2D(256, (3, 3), padding='same', strides=2))
model.add(PReLU())
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(PReLU())
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(PReLU())
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(PReLU())

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(PReLU())
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(PReLU())
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(PReLU())
model.add(Conv2D(16, (3, 3), padding='same'))
model.add(PReLU())
model.add(Conv2D(2, (3, 3), padding='same', activation='tanh'))
model.add(Conv2D(filters=3, kernel_size=(1, 1)))
model.add(PReLU())
model.add(UpSampling2D((2, 2)))

model.summary()

import tensorflow.keras.backend as K

def mse_loss(y_true, y_pred):
    return K.mean(K.square(y_true - y_pred))


def mean_gradient_error(y_true, y_pred):
    # Compute gradients of y_true and y_pred
    dy_true_dx = K.abs(y_true[:, :, :-1, :] - y_true[:, :, 1:, :])
    dy_pred_dx = K.abs(y_pred[:, :, :-1, :] - y_pred[:, :, 1:, :])
    dx_true_dy = K.abs(y_true[:, :-1, :, :] - y_true[:, 1:, :, :])
    dx_pred_dy = K.abs(y_pred[:, :-1, :, :] - y_pred[:, 1:, :, :])

    # Compute mean gradient error
    mge_loss = K.mean(K.abs(dy_true_dx - dy_pred_dx) + K.abs(dx_true_dy - dx_pred_dy))

    return mge_loss

def psnr(y_true, y_pred):
    max_pixel = 1.0
    mse = tf.keras.losses.mean_squared_error(y_true, y_pred)
    psnr_val = 10.0 * tf.math.log((max_pixel ** 2) / mse) / tf.math.log(10.0)
    return tf.where(tf.math.is_nan(psnr_val), tf.zeros_like(psnr_val), psnr_val)

def mse(y_true, y_pred):
    mse_val = tf.keras.losses.mean_squared_error(y_true, y_pred)
    return tf.where(tf.math.is_nan(mse_val), tf.zeros_like(mse_val), mse_val)

def pearson_similarity(y_true, y_pred):
    true_flat = tf.keras.backend.flatten(y_true)
    pred_flat = tf.keras.backend.flatten(y_pred)

    true_mean = tf.reduce_mean(true_flat)
    pred_mean = tf.reduce_mean(pred_flat)

    true_std = tf.keras.backend.std(true_flat)
    pred_std = tf.keras.backend.std(pred_flat)

    num = tf.reduce_mean((true_flat - true_mean) * (pred_flat - pred_mean))
    denom = true_std * pred_std

    similarity = num / denom
    return tf.where(tf.math.is_nan(similarity), tf.zeros_like(similarity), similarity)

from tensorflow.keras.losses import Loss

class CosineDissimilarityLoss(Loss):
    def __init__(self):
        super(CosineDissimilarityLoss, self).__init__()

    def call(self, y_true, y_pred):
        # Normalize the vectors
        y_true = tf.math.l2_normalize(y_true, axis=-1)
        y_pred = tf.math.l2_normalize(y_pred, axis=-1)

        # Calculate the cosine similarity
        cosine_similarity = tf.reduce_sum(y_true * y_pred, axis=-1)

        # Calculate the cosine dissimilarity loss
        cosine_dissimilarity_loss = 1 - cosine_similarity

        return cosine_dissimilarity_loss

loss_fn = CosineDissimilarityLoss()

def combined_loss(y_true, y_pred):
    # Calculate Mean Squared Error (MSE) loss
    mse_loss = tf.keras.losses.MSE(y_true, y_pred)

    # Calculate Mean Gradient Error (MGE) loss
    gradients_true = tf.image.image_gradients(y_true)
    gradients_pred = tf.image.image_gradients(y_pred)
    mge_loss = tf.keras.losses.MSE(gradients_true, gradients_pred)

    # Calculate Cosine Distance loss
    cosine_loss = tf.keras.losses.cosine_similarity(y_true, y_pred)
    cosine_distance = 1 - cosine_loss

    # Apply weights to the losses
    weighted_mse_loss = mse_loss * 1.0                 # Weight for MSE loss is 1.0
    weighted_mge_loss = mge_loss * 0.2                 # Weight for MGE loss is 0.1
    weighted_cosine_distance = cosine_distance * 1.0   # Weight for cosine distance loss is 1.0

    # Calculate combined loss
    combined_loss = weighted_mse_loss + weighted_mge_loss + weighted_cosine_distance
    return combined_loss

model.compile(loss=combined_loss, optimizer='adam',metrics=[psnr,mse,pearson_similarity])

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               verbose=1,
                               min_lr=0.5e-6)

callbacks = [lr_reducer]


history = model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=5000,
                      batch_size=32, callbacks=[lr_reducer])



In [ ]:
import cv2

print("\n       Input                        Ground Truth                  Predicted Value")
for i in range(3):

    r = random.randint(0, len(clean_frames)-1)

    x, y = blurry_frames[r],clean_frames[r]
    x_inp = x.reshape(1,128,128,3)
    result = model.predict(x_inp)
    result = result.reshape(128,128,3)

    fig = plt.figure(figsize=(12,10))
    fig.subplots_adjust(hspace=0.1, wspace=0.2)

    ax = fig.add_subplot(1, 3, 1)
    ax.imshow(x)

    ax = fig.add_subplot(1, 3, 2)
    ax.imshow(y)

    ax = fig.add_subplot(1, 3, 3)
    plt.imshow(result)


import pandas as pd
import matplotlib.pyplot as plt

# Plot training history
history_df = pd.DataFrame(history.history)
plt.figure(figsize=(10, 7))
history_df.plot()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training History')
plt.legend()
plt.show()